In [1]:
import data_process.self_supervised.batch_loader_v2 as batch_loader
import data_process.self_supervised.data_reader as data_reader
import data_process.self_supervised.label_generator as label_generator

In [2]:
path = "../train_80k"
dr = data_reader.Data_Reader(path)
lg = label_generator.LabelGenerator(dr)
bl = batch_loader.Batch_Loader_V2(dr, lg)

Computing Node Info : 100%|██████████| 82819/82819 [01:34<00:00, 875.07it/s]


In [3]:
def get_batch_pn_indices(itr_range):
        
        multi_pn = []
        for mask_index in itr_range:
            # declare the list to contain a positve subtree id and negative_sample_size neagtive subtree ids
            pn_list = []
            pn_list.append(bl.subtree_index[mask_index]) # add the positve subtree id at index 0

            # create the removal list
            which_label = bl.tree_mask_index[mask_index]
            label = bl.label_generator.labels[which_label]
            removal_list = bl.get_unique_label(label)
            
            # create negative subtree id list
            pn_list.extend(bl.select_negative_samples(removal_list))

            # append to dataset
            multi_pn.append(pn_list)
        return multi_pn

# import pickle

# with open('../batch_loader_v2/batch_loader_100k_test.pkl', 'wb') as file:
#     pickle.dump(bl, file)

In [4]:
import multiprocessing as mp
import math
if mp.cpu_count() > 2:
    # parallel process, greater than 2 because always reserve one core for other system usages
    num_cores = mp.cpu_count() - 2
    pool = mp.Pool(num_cores)

    # create the list of iterables
    itrs = []
    division = math.ceil(bl.num_train / num_cores)
    
    for i in range(num_cores):
        start = i * division
        end = start + division if start + division < bl.num_train else bl.num_train
        itrs.append(range(start, end))
        if (end == bl.num_train):
            break
    
    print(itrs)

    res = pool.map(get_batch_pn_indices, [itr_range for itr_range in itrs])

    for lis in res:
        bl.batch_pn_indices.extend(lis)
    print(len(bl.batch_pn_indices))

[range(0, 68298), range(68298, 136596), range(136596, 204894), range(204894, 273192), range(273192, 341490), range(341490, 409788), range(409788, 478086), range(478086, 546384), range(546384, 614682), range(614682, 682980), range(682980, 751278), range(751278, 819576), range(819576, 887874), range(887874, 956172), range(956172, 1024470), range(1024470, 1092768), range(1092768, 1161066), range(1161066, 1229364), range(1229364, 1297662), range(1297662, 1365960), range(1365960, 1434258), range(1434258, 1502556), range(1502556, 1570854), range(1570854, 1639152), range(1639152, 1707450), range(1707450, 1775723)]
1775723


In [5]:
pool.close()

In [2]:
import math
from tqdm import tqdm
import time
import pickle
with open('../batch_loader_v2/batch_loader_80k_5neg.pkl', 'wb') as file:
    pickle.dump(bl, file)


In [3]:
batch_size = 128
num_batches = math.ceil(bl.num_train/batch_size)
for idx in tqdm(range(num_batches)):
    bl.retrieve_batch(idx, batch_size)

100%|██████████| 13873/13873 [11:09<00:00, 20.72it/s]
